# Reforumlated Markowitz Model - Optimization Codes through Quantum Computing

Import the necessary libraries

In [24]:
import numpy as np
import pandas as pd
from dimod import ConstrainedQuadraticModel, Integer, Binary
import math
import time

Importing the Data

In [13]:
df = pd.read_excel("Cleaned_Data.xlsx")

Creating the training and testing dataframes

In [16]:
# Convert the 'date' column to datetime format 
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

In [18]:
# Splitting the DataFrame
training_end_date = '2019-12-31'
training_df = df[df.index <= training_end_date]
testing_df = df[df.index > training_end_date]

Calculating the expected returns and the variance covariance matrix

In [25]:
# Calculate expected profits for each asset
expected_profits = training_df.mean()

# Calculate the variance-covariance matrix
cov_matrix = training_df.cov()


#Converting the expected profits to a numpy array
expected_profits = expected_profits.values

#The number of assets
n_assets = len(expected_profits)


### Setting the limits of the constraints

Setting the other constraints limits

In [26]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
R = 1000

# Maximum units of each asset that can be purchased (D)
D = 50

# Maximum number of assets in the portfolio (K): The maximum number of different assets that can be included in the portfolio
K = 50

Creating the Decision Variables

In [27]:
y = [Binary(f"y_{i}") for i in range(n_assets)] #To represent y

In [28]:
X = [Integer(f"x_{i}") for i in range(n_assets)] #To represent x


Create the CQM Object

In [33]:
cqm = ConstrainedQuadraticModel()

Creating the Objective Function

In [34]:
H_obj = 0
for i in range(n_assets):
    H_obj -= expected_profits[i] * X[i]

Creating the risk constraint

In [38]:
# Convert the DataFrame to a NumPy array
cov_matrix_np = cov_matrix.to_numpy()

# Convert X to a NumPy array 
X_numeric = np.array(X)  

# Calculate the total variance
total_variance = np.dot(X_numeric.T, np.dot(cov_matrix_np, X_numeric))

Diversification Constraint

In [36]:
# Adjusting the diversification constraint
for i in range(n_assets):
    asset_diversification = X[i]
    cqm.add_constraint_from_model(asset_diversification, '<=', D, f"Diversification_{i}",weight=1)

Setting the Objective Function

In [37]:
cqm.set_objective(H_obj)

Setting the Risk Constraint

In [39]:
cqm.add_constraint_from_model(total_variance, '<=', R, "Variance", weight=1)

'Variance'

Adding the Linking Constraints

In [14]:
# Cardinality Constraints
for i in range(n_assets):
    cqm.add_constraint_from_model(X[i] - y[i] <= 0, label=f"Cardinality_{i}", weight=1)

Solving the optimization problem

In [16]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler(token="DEV-2559c1545adea2a38a589d00b09592efde0e1bd5")  

In [17]:
# Time the sampler
start_time = time.time()
sampleset = sampler.sample_cqm(cqm)
time_taken = time.time() - start_time

In [18]:
print(sampleset.first) 

Checking for the violation of constraints

In [17]:
for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
    print(label, violation)

Recreating the original decision variables

In [492]:
sample = sampleset.first[0]


# Number of decision variables
num_decision_variables = n_assets

# Reconstruct the original decision variables
decision_variables = []
for i in range(num_decision_variables):
    value = 0
    for j in range(M): 
        bit_value = sample[f'z_{i}_{j}']
        value += 2**j * bit_value
    decision_variables.append(value)

decision_variables


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0]

Calculating the total return

In [493]:
# Multiply corresponding elements and sum them up
total = sum(x * y for x, y in zip(expected_returns, decision_variables))

print(total)

0.08080462143532667


Now let us test to see if all the constraints are satisfied

In [494]:
# # Extract y values from sample
# y_optimized = np.array([sample[f'y_{i}'] for i in range(n_assets)])

# Convert decision_variables to a numpy array for easy manipulation
x_optimized = np.array(decision_variables)

# Now, checking the constraints with these values
# Constraint 1: Risk constraint
risk_value = np.sqrt(np.dot(np.dot(x_optimized.T, cov_matrix), x_optimized))
print(f"Risk Constraint Satisfied: {risk_value <= R}")
print(f"Risk Value: {risk_value} <= {R}")

# # Constraint 2: Diversification and linking constraint
# diversification_values = x_optimized <= D * y_optimized
# print(f"Diversification and Linking Constraints Satisfied: {all(diversification_values)}")

# # Constraint 3: Cardinality constraint
# cardinality_value = np.sum(y_optimized)
# print(f"Cardinality Constraint Satisfied: {cardinality_value <= K}")
# print(f"Number of Assets Included: {cardinality_value} <= {K}")

# Constraint 4: Non-negativity constraint
non_negativity_satisfied = all(x_optimized >= 0)
print(f"Non-negativity Constraints Satisfied: {non_negativity_satisfied}")


Risk Constraint Satisfied: True
Risk Value: 0.09989719898711656 <= 0.1


Checking the assets to invest in

In [495]:
values_list = x_optimized.tolist()

# Creating a new DataFrame with column names and values where values are > 0
filtered_data = {'Asset': [], 'Value': []}
for column_name, value in zip(df.columns, values_list):
    if value > 0:
        filtered_data['Asset'].append(column_name)
        filtered_data['Value'].append(value)

new_df = pd.DataFrame(filtered_data)

print(new_df)

  Asset  Value
0   CDW    3.0
1  FANG    1.0
2     V    1.0


Converting the dataframe into an excel

In [496]:
# new_df.to_excel("DF1.xlsx")

Saving the Results

In [497]:
# B = [time_taken, risk_value, R, total]
# A.append(B)

# # Create a DataFrame with the specified column names
# df = pd.DataFrame(A, columns=['time_taken', 'risk_value', 'R', 'total'])

# df.to_excel("Quantum_Results.xlsx")